In [ ]:
%env THEANO_FLAGS=device=gpu5,floatX=float32,exception_verbosity=high,lib.cnmem=0.95,mode=FAST_RUN
%run ./dimanet-critic.ipynb

env: THEANO_FLAGS=device=gpu5,floatX=float32,exception_verbosity=high,lib.cnmem=0.95,mode=FAST_RUN


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 5: GeForce GTX 1080 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/anaconda3/lib/python3.5/site-packages/agentnet/utils/logging.py:14: UserWarning: [Verbose>=1] You called get_output from recurrence several times without gathering the updates.
(A) If you wanted to get two outputs from recurrence, use NOT
>>>out1 = get_output(rec[layer1])
>>>out2 = get_output(rec[layer2])
but instead:
>>>out1,out2 = get_output((rec[layer1],rec[layer2])) #or rec[layer1,layer2].
(B) If you want to run recurrence several times and accumulate updates from all runs,use get_output(...,accumulate_updates=True) to silence the warning.
(C) If you want to get rid of old updates, u

Some examples ['cocksuka', 'doggiestyle', 'dildo', 'fooker', 'unwieldy', 'biatch', 'flange', 'doosh', 'damn', 'n1gger']
Number of target words 236


In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

test_phrases = [['Hello! How are you?'],
                ['How old are you?'],
                ['Are you fucking kidding me?'],
                ['Suck. What are you doing?'], 
                ['You are piece of shit!!!'], 
                ['holy fucking crap. you are motherfucker']]

test_phrases_ids = phrase2matrix(test_phrases)

from agentnet.utils.persistence import save,load
load(Gen.recurrence, './weights/LM2.pkl')

BATCH_SIZE = 64
TRAINING_TIME_IN_BATCHES = 500
VERBOSITY = 10 # number of batches before printing
NUM_BATCHES = len(contexts)//BATCH_SIZE
LLH_ALPHA = [90.0, 70.0, 50.0, 30.0, 20.0]
CHANGE_ALPHA_EVERY = 100
SAVE_EVERY = 100
MODELS_SUFFIX = '_bepolite_my_lm'

AT.LLH_ALPHA.set_value(LLH_ALPHA[0])

loss_history = {'critic':[], 'pg_loss':[], 'llh_loss':[]}
rewards_history = []
for nb, (batch_context, batch_answers) in enumerate(iterate_minibatches(contexts, BATCH_SIZE)):
    if (nb + 1) >= TRAINING_TIME_IN_BATCHES:
        print("TRAINING FINISHED!")
        break
    
    llh_alpha = LLH_ALPHA[min(int(nb/CHANGE_ALPHA_EVERY), len(LLH_ALPHA)-1)]
    AT.LLH_ALPHA.set_value(llh_alpha)
    
    ## Training stuff.
    pg_loss, llh_loss, policy, actions, advantage, rewards, is_alive = AT.train_step(batch_context, batch_answers)
    critic_loss, _, _, _, _ = CT.train_step(batch_context)

    loss_history['critic'].append(critic_loss)
    loss_history['pg_loss'].append(pg_loss)
    loss_history['llh_loss'].append(llh_loss)
    rewards_history.append(rewards.sum(axis=1).mean())

    #### PRINTING STUFF. ####
    if (nb + 1) % VERBOSITY == 0:
        clear_output(wait=True) 

        # Visualize answer with maximum reward.
        idx = np.argmax((rewards*is_alive).sum(axis=1))
        ans_seq, ans_rewards, ans_advantage = actions[idx], rewards[idx], advantage[idx]
        answer_len = list(ans_seq).index(EOS_ix) if EOS_ix in ans_seq else len(ans_seq)

        ans = [tokens[i] for i in ans_seq][:answer_len+1]
        inp_phrase = [tokens[i] for i in batch_context[idx][:list(batch_context[idx]).index(EOS_ix)]]
        corr_ans = [tokens[i] for i in batch_answers[idx][:list(batch_answers[idx]).index(EOS_ix)]]
        print("Most rewarded answer: ", ans)
        print("Input phrase: ", inp_phrase)
        print("Correct answer: ", corr_ans)
        plt.scatter(range(len(ans_rewards[:answer_len+1])), ans_rewards[:answer_len+1])
#             plt.plot(ans_V_pred[:answer_len+1], label='predicted')
#             plt.plot(ans_V_ref[:answer_len+1], label='reference')
        plt.plot(ans_advantage[:answer_len+1], label='A(s,a)')
        plt.ylim(ymin=0)
        plt.yticks(np.arange(10))
        plt.legend()
        plt.show()

        # Answering on test phrases.
        print("Answers on test phrases:")
        words_seq, critic_values, _, _ = Critic.predict(test_phrases_ids)
        answers = []
        for i in range(words_seq.shape[0]):
            answer = words_seq[i]
            if EOS_ix in answer:
                answer = answer[:list(answer).index(EOS_ix)]
            answers.append([tokens[idx] for idx in answer])

        print("Processed {}/{} batches in current epoch".format(nb+1, NUM_BATCHES))
        print("LLH_ALPHA = {0:.3f}".format(float(AT.LLH_ALPHA.get_value())))
        print("Critic Loss (averaged with last 10 batches): {0:.5f}".format(np.mean(loss_history['critic'][-10:])))
#             print("Actor Loss (averaged with last 10 batches): {0:.5f}".format(np.mean(loss_history['actor'][-10:])))
        print("Avg reward: {0:.5f}".format(np.mean(rewards_history[-10:])))
        print("Loss history:")
        plt.plot(loss_history['critic'], label='critic')
        plt.legend()
        plt.show()
        plt.plot(loss_history['pg_loss'], label='pg_loss')
        plt.plot(loss_history['llh_loss'], label='llh_loss')
        plt.legend()
        plt.show()

        for i in range(len(test_phrases)):
            print("Phrase:\t", test_phrases[i])
            answer_tok_critic_value = zip(answers[i], critic_values[i])
            print("Answer:\t",)
            for tok,value in answer_tok_critic_value:
                print('"{}"({:.3f}) '.format(tok, value),)
        print('---'*5)
        print("Answers on 'hello world!':")
        for i in range(5):
            print(Gen.reply('hello world!'))

    if nb % SAVE_EVERY == 0:
        criticfname = './weights/critic{}{}.pkl'.format(MODELS_SUFFIX, LLH_ALPHA)
        actorfname = './weights/actor{}{}.pkl'.format(MODELS_SUFFIX, LLH_ALPHA)
        print("Save actor and critic weights to '{}' and '{}'".format(criticfname, actorfname))
        save(Critic.l_critic_values, criticfname)
        save(Gen.recurrence, actorfname)

# Reward & LLH checking

In [3]:
from agentnet.utils.persistence import load
get_rewards = theano.function([Enc.input_phrase], [Critic.rewards, Gen.words_seq], allow_input_downcast=True, no_default_updates=True)

In [12]:
# load(Gen.recurrence, './weights/LM2.pkl')

In [29]:
TEMPERATURE.set_value(1.0)

In [27]:
for _ in range(10):
    print(Gen.reply('how old are you?'))

you are ?
you ' re lucky .
eight .
nineteen .
i ' m 10 .
that ' s what you call mentions me too .
i don ' t know .
is this your son ?
have you ?
five hundred ?


In [5]:
for _ in range(10):
    print(Gen.reply('get the fuck out!'))

this is our dance fooled tonight !
what ?
_UNK_ , my dear _UNK_ , i ' m a racist . "
it doesn ' t matter .
sorry i dragged you along .
did he go to jail ?
hello .
who did it ?
get a nice my from the front .
does that look okay ?


In [4]:
def get_rewards_and_llh(batch_size=64, num_batches=10000):
    rewards = []
    llhs = []

    for nb, (batch_context, batch_answers) in tqdm.tqdm_notebook(enumerate(iterate_minibatches(contexts, batch_size))):
        if nb <= TRAINING_TIME_IN_BATCHES:
            continue
        if nb-TRAINING_TIME_IN_BATCHES >= num_batches:
#         if nb >= num_batches:
            break
        b_rewards, b_answers = get_rewards(batch_context)
        rewards.extend(list(b_rewards.sum(axis=1)))
        llhs.append(GenTrain.get_llh(batch_context, batch_answers))
    return rewards, llhs

In [5]:
rewards, llhs = get_rewards_and_llh(64, 2000)

In [6]:
np.exp(np.mean(llhs)), np.mean(rewards)

(3.2018166, -0.050306403201600798)

### Now calculate reward over batches where input contains obscene.

In [ ]:
BATCH_SIZE = 400
NUM_BATCHES = 1000
rewards = []


for nb, (batch_context, batch_answers) in tqdm.tqdm_notebook(enumerate(iterate_minibatches(contexts, BATCH_SIZE))):
    if nb >= NUM_BATCHES:
        break
    mask = (batch_context[:, :, None] == np.array(list(target_idxs))[None, None, :]).any(-1).any(-1)
    bb = batch_context[mask]
        
    b_rewards, b_answers = get_rewards(bb)
    rewards.extend(list(b_rewards.sum(axis=1)))

In [ ]:
np.mean(rewards)

----

In [12]:
target_words

{'5h1t',
 '5hit',
 'Eskimo',
 'Indian',
 'Jerry',
 'Kraut',
 'LEN',
 'Paddy',
 'Paki',
 'Pygmy',
 'Taffy',
 'WASP',
 'Yank',
 'Yankee',
 'a2m',
 'a55',
 'a_s_s',
 'adult',
 'amateur',
 'anal',
 'anilingus',
 'anus',
 'ar5e',
 'arrse',
 'arse',
 'arsehole',
 'ass',
 'ass-fucker',
 'ass-hat',
 'ass-hole',
 'ass-jabber',
 'ass-pirate',
 'assbag',
 'assbandit',
 'assbanger',
 'assbite',
 'assclown',
 'asscock',
 'asscracker',
 'asses',
 'assface',
 'assfuck',
 'assfucker',
 'assfukka',
 'assgoblin',
 'asshat',
 'asshead',
 'asshole',
 'assholes',
 'asshopper',
 'assjacker',
 'asslick',
 'asslicker',
 'assmonkey',
 'assmucus',
 'assmunch',
 'assmuncher',
 'assnigger',
 'asspirate',
 'assshit',
 'assshole',
 'asssucker',
 'asswad',
 'asswhole',
 'asswipe',
 'autoerotic',
 'axwound',
 'b!tch',
 'b00bs',
 'b17ch',
 'b1tch',
 'ballbag',
 'ballsack',
 'bampot',
 'bangbros',
 'bareback',
 'bastard',
 'beaner',
 'beastial',
 'beastiality',
 'bellend',
 'bestial',
 'bestiality',
 'bi+ch',
 'biatch'